### TODO 
- Train a specific BERT before using it
- Add a step to use an LLM (probably LLnan3 locally) to add features, labels etc

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from helper_functions.topic_modelling.flatten_articles import flatten_articles
from helper_functions.topic_modelling.sentiment_analysis import perform_sentiment_analysis
from helper_functions.topic_modelling.add_counts_columns_parallel import (
    add_synonym_frequency_columns,
    add_category_count_columns
)
from helper_functions.topic_modelling.aggregate_articles import aggregate_articles, FS_CATEGORIES

# 1️⃣ Load enriched event dataset with:
#    - articles (list of cleaned + truncated text strings)
#    - NER_admin0_list / admin1 / admin2
#    - event metadata (ADMIN0/1/2, CS_score, period, etc.)
# ================================================================

exploded_df = pd.read_parquet("../data/gdelt/gkg/scraped_urls/cleaned_filtered_urls.parquet")

# ================================================================
# 3️⃣ (OPTIONAL SAFETY) Ensure clean_text is string
# ================================================================
exploded_df["clean_text"] = exploded_df["clean_text"].astype(str)

# ================================================================
# 4️⃣ Sentiment Analysis
# ================================================================
exploded_df = perform_sentiment_analysis(exploded_df, text_col="clean_text")
print("Sentiment analysis done.")

# ================================================================
# 5️⃣ Keyword frequency & category count features
# ================================================================
exploded_df = add_synonym_frequency_columns(exploded_df, text_col="clean_text")
print("Frequency counts done")
exploded_df = add_category_count_columns(exploded_df, text_col="clean_text")
print("Keyword & category counts added.")

# ================================================================
# 6️⃣ Save in chunks
# ================================================================
out_dir = "../data/gdelt/gkg/5_modelled"
os.makedirs(out_dir, exist_ok=True)
exploded_df.to_parquet(out_dir + "/gkg_exploded_with_counts.parquet", index=False)

print("✅ Processing complete.")
exploded_df.head()

In [ ]:
def clean_array_columns(df):
    for col in df.columns:
        # convert ["2020-02-13"] → "2020-02-13"
        df[col] = df[col].apply(
            lambda x: x[0] if isinstance(x, (list, tuple, np.ndarray)) and len(x) == 1 else x
        )
    return df

exploded_df = clean_array_columns(exploded_df)


In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertPreTrainedModel, BertModel
import transformers
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from scipy.stats import mode

# ────────────────
# CONFIGURATION
# ────────────────
MODEL_DIR = "./for_bert/bert_multitask_model"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 512
STRIDE = 256

# These must exactly match training
impact_classes = [
    "death", "injury", "displacement", "missing", "disease",
    "economic_loss", "conflict", "weather_shock", "policy_change", "none"
]
urgency_classes = ["low", "moderate", "high", "unclear"]
resource_cols = [
    "resource_food", "resource_water", "resource_cash_aid", "resource_healthcare", "resource_shelter",
    "resource_livelihoods", "resource_education", "resource_infrastructure", "resource_none"
]

# ────────────────
# LABEL ENCODERS
# ────────────────
impact_enc = LabelEncoder()
impact_enc.fit(impact_classes)

urgency_enc = LabelEncoder()
urgency_enc.fit(urgency_classes)


# ────────────────
# MODEL DEFINITION
# ────────────────
class MultiTaskBERT(BertPreTrainedModel):
    def __init__(self, config, num_impact, num_urgency, num_resource):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.1)
        self.impact_head = nn.Linear(config.hidden_size, num_impact)
        self.urgency_head = nn.Linear(config.hidden_size, num_urgency)
        self.resource_head = nn.Linear(config.hidden_size, num_resource)
        self.init_weights()
    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        pooled = self.dropout(self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output)
        return {
            "impact": self.impact_head(pooled),
            "urgency": self.urgency_head(pooled),
            "resource": self.resource_head(pooled)
        }

# ────────────────
# LOAD MODEL & TOKENIZER
# ────────────────
tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
model = MultiTaskBERT.from_pretrained(
    MODEL_DIR,
    num_impact=len(impact_classes),
    num_urgency=len(urgency_classes),
    num_resource=len(resource_cols)
)
model.to(DEVICE)
model.eval()

# ────────────────
# CHUNKING FUNCTION (from your training)
# ────────────────
transformers.logging.set_verbosity_error()  # At the top to silence all warnings
def chunk_input_ids(text, tokenizer, max_length=512, stride=256):
    # Get raw tokens (no special tokens)
    tokens = tokenizer.encode(text, add_special_tokens=False)
    max_len_no_special = max_length - 2
    chunks = []
    for i in range(0, len(tokens), stride):
        chunk = tokens[i:i + max_len_no_special]
        # Now, turn chunk back into string and re-encode with special tokens, pad/truncate to max_length
        chunk_text = tokenizer.decode(chunk)
        inputs = tokenizer(
            chunk_text,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        chunks.append(inputs)
    return chunks

def predict_for_text(text):
    chunks = chunk_input_ids(str(text), tokenizer, max_length=MAX_LENGTH, stride=STRIDE)
    if not chunks:
        # fallback: [UNK] padded to 512
        inputs = tokenizer(
            tokenizer.unk_token,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        chunks = [inputs]
    input_ids_tensor = torch.cat([chunk['input_ids'] for chunk in chunks]).to(DEVICE)
    attention_mask_tensor = torch.cat([chunk['attention_mask'] for chunk in chunks]).to(DEVICE)

    # --- DEBUG & CATCH ---
    assert input_ids_tensor.shape[1] <= 512, f"Found input_ids length {input_ids_tensor.shape[1]}"

    with torch.no_grad():
        outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor)
    # ... (rest of your code unchanged)
    impact_preds = outputs["impact"].argmax(dim=1).cpu().numpy()
    urgency_preds = outputs["urgency"].argmax(dim=1).cpu().numpy()
    resource_probs = torch.sigmoid(outputs["resource"]).cpu().numpy()
    resource_preds = (resource_probs > 0.5).astype(int)

    impact_final = mode(impact_preds, keepdims=True)[0][0]
    urgency_final = mode(urgency_preds, keepdims=True)[0][0]
    resource_final = (resource_preds.sum(axis=0) > 0).astype(int)

    impact_label = impact_enc.inverse_transform([impact_final])[0]
    urgency_label = urgency_enc.inverse_transform([urgency_final])[0]
    resource_dict = {col: int(val) for col, val in zip(resource_cols, resource_final)}
    return impact_label, urgency_label, resource_dict

# ────────────────
# APPLY TO DATAFRAME
# ────────────────

# Example: df should have a column 'clean_text'
tqdm.pandas()  # progress bar

exploded_df = pd.read_parquet("../data/gdelt/events/5_modelled/exploded_partial.parquet")

results = exploded_df['clean_text'].progress_apply(predict_for_text)

# Add predictions to dataframe
exploded_df['pred_impact_type'] = results.apply(lambda x: x[0])
exploded_df['pred_urgency'] = results.apply(lambda x: x[1])
for i, col in enumerate(resource_cols):
    exploded_df[f'pred_{col}'] = results.apply(lambda x: x[2][col])

exploded_df.to_parquet("../data/gdelt/events/5_modelled/exploded_full.parquet")
exploded_df.head(20)

In [ ]:
import pandas as pd
events = pd.read_parquet("../data/gdelt/events/5_modelled/exploded_full.parquet", engine="fastparquet")
events = events.drop(columns=[c for c in events.columns if c.startswith("sentiment.")])
events.head()

In [ ]:
# 8) Re-aggregate so each original row has arrays of new info
group_cols = ['ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score', 'period']
original_list_cols = ['SQLDATE', 'EventCode', 'SOURCEURL', 'NumMentions', 'NumSources', 'NumArticles']

final_df = aggregate_articles(
    exploded_df,
    group_cols=group_cols,
    original_list_cols=original_list_cols,
    original_df=original_df
)


# Create a subset for topic modeling
bertopic_df = exploded_df[valid_rows].copy()
print(f"BERTopic will run on {bertopic_df.shape[0]} articles (filtered from {exploded_df.shape[0]})")

# Run BERTopic only on valid rows
bertopic_df, topic_model = topic_modeling_bertopic(bertopic_df, text_col='clean_text', language='english')

# 7) Analyze Topics & Sentiment
conflict_keywords = [
    'conflict', 'war', 'violence', 'military', 'coup', 'insurgency', 'terrorism',
    'battle', 'hostility', 'clash', 'unrest'
]
climate_keywords = [
    'climate', 'weather', 'drought', 'flood', 'rainfall', 'heatwave',
    'temperature anomaly', 'extreme weather'
]
food_security_keywords = [
    'food security', 'economic shock', 'inflation', 'poverty', 'supply chain',
    'food prices', 'commodity prices', 'subsistence', 'rural development',
    'agricultural production', 'sustainable agriculture', 'food access',
     'food aid', 'emergency relief', 'food assistance', 'nutrition', 'hunger', 'famine',
    'malnutrition', 'crop yield', 'harvest', 'farming', 'livestock'
]

if topic_model is None:
    print("Skipping analyze_topics_and_sentiment due to lack of a topic model.")
else:
    bertopic_df = analyze_topics_and_sentiment(bertopic_df, topic_model, conflict_keywords, climate_keywords, food_security_keywords)
print("Analysis of topics and sentiment completed")

exploded_df = exploded_df.merge(bertopic_df[['topic', 'topic_probability', 'topic_label', 'severity']], how='left', left_index=True, right_index=True)
print("Topic modeling and sentiment analysis done.")

# 8) Re-aggregate so each original row has arrays of new info
group_cols = ['ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score', 'period']
original_list_cols = ['SQLDATE', 'EventCode', 'SOURCEURL', 'NumMentions', 'NumSources', 'NumArticles', 'headers', 'bodies']

final_df = aggregate_articles(
    exploded_df,
    group_cols=group_cols,
    original_list_cols=original_list_cols,
    original_df=original_df
)

# 9) Save or proceed with analysis
keep_cols = [
    'ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score', 'period',
    'SQLDATE', 'EventCode', 'SOURCEURL', 'NumMentions',	'NumSources', 'NumArticles',
    'headers', 'sentiment', 'compound_score',
    'topic', 'topic_probability', 'topic_label', 'severity',
    'named_entities', 'number_context', 'TotalNumMentions', 'TotalNumSources', 'TotalNumArticles'
]
for cat in FS_CATEGORIES:
    keep_cols.append(f"{cat}_count")
    keep_cols.append(f"{cat}_freq")
    keep_cols.append(f"{cat}_sentences")

keep_cols = [c for c in keep_cols if c in final_df.columns]
final_df = final_df[keep_cols]


final_df.to_parquet('../data/gdelt/events/5_modelled/data.parquet')

final_df.head()

In [ ]:
exploded_df.applymap(lambda x: isinstance(x, (list, tuple, np.ndarray))).any()


In [ ]:
exploded_df.head()

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_parquet("../data/gdelt/events/5_modelled/exploded_partial_chunk_1.parquet")
df.head()

# aggregate everything at the end


In [ ]:
import os
import re
import numpy as np
import pandas as pd

# ----------------------- Utility Function -----------------------

def clean_text_array(arr):
    """Remove control characters from each string in a list/array."""
    if isinstance(arr, (list, np.ndarray)):
        return [
            re.sub(r'[\x00-\x1F\x7F-\x9F]', '', str(x))
            for x in arr if isinstance(x, str) and x.strip()
        ]
    return arr

# ----------------------- Load GDELT Final Output -----------------------

mapped_path = "../data/gdelt/events/3_mapped/gdelt_mapped.parquet"
if not os.path.exists(mapped_path):
    raise FileNotFoundError(f"Missing mapped GDELT file: {mapped_path}")

df_final = pd.read_parquet(mapped_path)
print("✅ Loaded GDELT mapped dataset:", len(df_final))

# ----------------------- Build ADMIN lookup from valid rows -----------------------

locations = df_final[['ADMIN0', 'ADMIN1', 'ADMIN2']]
locations = locations[locations['ADMIN0'].notna()]
locations = locations.drop_duplicates()

# Dictionary: (ADMIN1, ADMIN2) → ADMIN0
admin_lookup = {
    (row.ADMIN1, row.ADMIN2): row.ADMIN0
    for _, row in locations.iterrows()
}

def fill_admin0(row):
    """Fill missing ADMIN0 using (ADMIN1, ADMIN2) lookup."""
    if pd.isna(row['ADMIN0']):
        return admin_lookup.get((row['ADMIN1'], row['ADMIN2']), None)
    return row['ADMIN0']

# Apply the fill
df_final['ADMIN0'] = df_final.apply(fill_admin0, axis=1)

print("🧭 Missing ADMIN0 after filling:", df_final['ADMIN0'].isna().sum())

# ----------------------- Prepare Mapping from Scraped URLs -----------------------

# Ensure df_scraped has needed columns
required_cols = ['url', 'clean_text', 'NER_admin0', 'NER_admin1', 'NER_admin2']
assert all(col in df_scraped.columns for col in required_cols), \
    f"df_scraped must contain {required_cols}, found: {df_scraped.columns.tolist()}"

# Create mappings from URL → clean_text / NER
url_to_cleantext = df_scraped.set_index('url')['clean_text'].to_dict()
url_to_admin0 = df_scraped.set_index('url')['NER_admin0'].to_dict()
url_to_admin1 = df_scraped.set_index('url')['NER_admin1'].to_dict()
url_to_admin2 = df_scraped.set_index('url')['NER_admin2'].to_dict()

def extract_list(urls, mapping):
    """Extract a list of mapped values corresponding to a list of SOURCEURL URLs."""
    if isinstance(urls, (list, np.ndarray)):
        return [mapping[u] for u in urls if isinstance(u, str) and u in mapping]
    return []

# ----------------------- Map Clean Text + NER to df_final -----------------------

df_final['articles'] = df_final['SOURCEURL'].apply(
    lambda urls: extract_list(urls, url_to_cleantext)
)

df_final['NER_admin0_list'] = df_final['SOURCEURL'].apply(
    lambda urls: extract_list(urls, url_to_admin0)
)

df_final['NER_admin1_list'] = df_final['SOURCEURL'].apply(
    lambda urls: extract_list(urls, url_to_admin1)
)

df_final['NER_admin2_list'] = df_final['SOURCEURL'].apply(
    lambda urls: extract_list(urls, url_to_admin2)
)

# Clean control characters in articles
df_final['articles'] = df_final['articles'].apply(clean_text_array)

# ----------------------- Sanity Checks -----------------------

df_final['num_articles'] = df_final['articles'].apply(len)
print("🧾 Rows with no matched articles:", (df_final['num_articles'] == 0).sum())
print("📊 Distribution of article counts per event:")
print(df_final['num_articles'].describe())

# ----------------------- Save Full File -----------------------

out_dir = "../data/gdelt/events/4_scraped"
os.makedirs(out_dir, exist_ok=True)

df_final = df_final[
    (df_final["num_articles"] > 0) |
    (df_final["CS_score"].notna())
].reset_index(drop=True)

full_out = os.path.join(out_dir, "data_with_articles_full.parquet")
df_final.to_parquet(full_out, index=False)
print(f"💾 Saved full enriched dataset: {full_out}")

# ----------------------- Save in Chunks -----------------------

chunk_size = 20_000
num_chunks = (len(df_final) + chunk_size - 1) // chunk_size

for i in range(num_chunks):
    start_row = i * chunk_size
    end_row = min((i + 1) * chunk_size, len(df_final))
    chunk_df = df_final.iloc[start_row:end_row].copy()
    chunk_path = os.path.join(out_dir, f"data_with_articles_chunk_{i}.parquet")
    chunk_df.to_parquet(chunk_path, index=False)
    print(f"✅ Saved chunk {i+1}/{num_chunks}: {chunk_path}")

print(f"\n🎉 Done! {num_chunks} Parquet chunk files saved in {out_dir}")